# Módulos add and norm y feed forward finales

Solo nos quedan los módulos `Add & Norm` y `Feed Forwar`

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_add_and_norm4.png" alt="Add and norm" style="width:425px;height:626px;">
  <img src="Imagenes/transformer_architecture_model_decoder_feed_forward.png" alt="Encoder Feed Forward" style="width:425px;height:626px;">
  <img src="Imagenes/transformer_architecture_model_add_and_norm5.png" alt="Add and norm" style="width:425px;height:626px;">
</div>

Como ya los hemos explicado, estos tres últimos los volvemos a contar en un único notebook

## Add & Norm

Estos módulos aportan beneficios a la red por las conexiones residuales

## Feed Forward

Consiste en una red fully conected como las que hemos visto al principio del curso, pero de dos capas, exactamente en el paper lo definen así

<div style="text-align:center;">
  <img src="Imagenes/feed_forward_equation.png" alt="Feed Forward equation">
</div>

Esta fórmula lo que nos indica es que hay que pasar la matriz por una capa lineal con pesos $W_1$ y $b_1$, aplicarle una función de activación Relu y a continuación pasarle por otra capa lineal con pesos $W_2$ y $b_2$

El motivo de esta capa es añadir una no linealidad al encoder, lo cual trae las siguientes ventajas

 * Incremento de la capacidad del modelo:
 * Extracción de características:

En la primera capa del `Feed Forward` se podrá aumentar la dimensión de la matriz, pero a la salida del módulo `Feed Forward` la dimensión de la matriz tiene que ser $\left(m_E \times n_E\right)$.

En el paper proponen que si la matriz de entrada tiene una dimensión de embedding de 512, a la salida de la primera capa la matriz tenga una dimensión de embedding de 2048 y a la salida de la segunda capa vuelva a tener una dimensión de embedding de 512. Por lo que en la parte de `Feed Forward` internamente se multiplica la dimensión de la matriz por 4 y luego se vuelve a reducir

## Implementación

Recuperamos las clases que ya hemos hecho antes

In [1]:
import torch
import torch.nn as nn

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

In [2]:
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

Como vamos a necesitar la salida del encoder volvemos a escribir todas las clases del encoder

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        """
        Args:
            vocab_size: size of vocabulary
            embed_dim: dimension of embeddings
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            out: embedding vector
        """
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super().__init__()
        self.embedding_dim = embedding_model_dim

        # create constant 'positional_encoding' matrix with values dependant on pos and i
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = math.sin(pos / (10000 ** ((2 *     i) / self.embedding_dim)))
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim)))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
        # make embeddings relatively larger
        x = x * math.sqrt(self.embedding_dim)
        
        # add encoding matrix to embedding (x)
        sequence_len = x.size(1)
        # x = x + torch.autograd.Variable(self.positional_encoding[:,:sequence_len], requires_grad=False)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, query, key, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / math.sqrt(self.dim_embedding)
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.nn.functional.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector
            mask: mask matrix (optional)

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V, mask=mask)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        multi_head_attention = self.multi_head_attention(x, x, x)
        add_and_norm_1 = self.add_and_norm_1(x, multi_head_attention)
        feed_forward = self.feed_forward(add_and_norm_1)
        add_and_norm_2 = self.add_and_norm_2(add_and_norm_1, feed_forward)
        return add_and_norm_2

class Encoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx):
        super().__init__()
        self.encoder_layers = nn.ModuleList([EncoderLayer(heads, dim_embedding) for _ in range(Nx)])
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, dim_embedding, max_sequence_len, heads, Nx):
        super().__init__()
        self.input_embedding = Embedding(vocab_size, dim_embedding)
        self.positional_encoding = PositionalEncoding(max_sequence_len, dim_embedding)
        self.encoder = Encoder(heads, dim_embedding, Nx)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        input_embedding = self.input_embedding(x)
        positional_encoding = self.positional_encoding(input_embedding)
        encoder = self.encoder(positional_encoding)
        return encoder


Volvemos a definir la función que obtiene el embbeding más el positional encoding de BERT

In [4]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Creamos una sentencia para el encoder, ya que ahora va a entrar una sentencia al encoder y otra al decoder

In [5]:
sentence_encoder = "I gave the dog a bone because it was hungry"
tokens_encoder, input_ids_encoder, token_embeddings_encoder, positional_encodings_encoder, embeddings_with_positional_encoding_encoder = extract_embeddings(sentence_encoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Creamos un objeto `encoder`

In [6]:
vocab_size = 30522
dim_embedding = token_embeddings_encoder.shape[-1]
max_sequence_len = token_embeddings_encoder.shape[1]
heads = 8
Nx = 6
transformer_encoder = TransformerEncoder(vocab_size, dim_embedding, max_sequence_len, heads, Nx)

Obtenemos la salida del transformer encoder

In [7]:
encoder_output = transformer_encoder(input_ids_encoder)
encoder_output.shape

torch.Size([1, 12, 768])

Ahora generamos una sentencia para el decoder, lo que haremos será generar la sentencia del encoder traducida al español

In [8]:
sentence_encoder = "I gave the dog a bone because it was hungry"
sentence_decoder = "Le di un hueso al perro porque tenía hambre"
tokens_decoder, input_ids_decoder, token_embeddings_decoder, positional_encodings_decoder, embeddings_with_positional_encoding_decoder = extract_embeddings(sentence_decoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Instanciamos un objeto de la clase `Multi-Head Attention` enmascarada y obtenemos su salida

In [9]:
def create_mask(sequence_len):
    """
    Args:
        sequence_len: length of sequence
        
    Returns:
        mask matrix
    """
    mask = torch.tril(torch.ones((sequence_len, sequence_len)))
    return mask
sequence_len = input_ids_decoder.shape[1]
mask = create_mask(sequence_len)
mask.shape

torch.Size([16, 16])

In [10]:
dim_embedding = embeddings_with_positional_encoding_decoder.shape[-1]
heads = 8
masked_multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)

K = embeddings_with_positional_encoding_decoder
V = embeddings_with_positional_encoding_decoder
Q = embeddings_with_positional_encoding_decoder
masked_attention = masked_multi_head_attention(Q, K, V, mask=mask)
masked_attention.shape

torch.Size([1, 16, 768])

Creamos ahora un objeto de la clase `Add & Norm` y calculamos su salida

In [11]:
add_and_norm_3 = AddAndNorm(dim_embedding)
masked_attention_add_and_norm = add_and_norm_3(embeddings_with_positional_encoding_decoder, masked_attention)
masked_attention_add_and_norm.shape

torch.Size([1, 16, 768])

Instanciamos un objeto de la clase `ENcoder-Decoder Multi-Head Attention` y obtenemos su salida

In [12]:
encoder_decoder_multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)
K = encoder_output
V = encoder_output
Q = masked_attention_add_and_norm

encoder_decoder_attention = encoder_decoder_multi_head_attention(Q, K, V)
encoder_decoder_attention.shape

torch.Size([1, 16, 768])

Creamos ahora un objeto de la clase `Add & Norm` y calculamos su salida

In [13]:
add_and_norm_4 = AddAndNorm(dim_embedding)
encoder_decoder_attention_add_and_norm = add_and_norm_4(masked_attention_add_and_norm, encoder_decoder_attention)
encoder_decoder_attention_add_and_norm.shape

torch.Size([1, 16, 768])

Creamos ahora un objeto de la clase `Feed Forward` y calculamos su salida

In [14]:
feed_forward = FeedForward(dim_embedding)
feed_forward_output = feed_forward(encoder_decoder_attention_add_and_norm)
feed_forward_output.shape

torch.Size([1, 16, 768])

Creamos ahora un objeto de la clase `Add & Norm` y calculamos su salida

In [15]:
add_and_norm_5 = AddAndNorm(dim_embedding)
add_and_norm_5_output = add_and_norm_5(encoder_decoder_attention_add_and_norm, feed_forward_output)
add_and_norm_5_output.shape

torch.Size([1, 16, 768])